## Library Widgets for Graph Exploration

### Load an RDF graph

In this example, we will use the `ipyradiant` `FileManager`.

In [ ]:
from ipyradiant import FileManager, PathLoader

lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
lw

In [ ]:
from ipyradiant.visualization.explore import GraphExploreNodeSelection, RDFSubjectSelectMultiple, RDFTypeSelectMultiple

In [ ]:
ts = RDFTypeSelectMultiple(label="Set custom label (e.g. Available types):")
ts.graph = lw.graph
ts

In [ ]:
ts.selections  # These are the URIs of the selected objects

TODO how to index so that we can do something like `type_select.value = [something here]`

For now, choose `voc:Droid` and `voc:Film`.

In [ ]:
ss = RDFSubjectSelectMultiple(label="Set custom label (e.g. Available subjects):")
ss.graph = lw.graph
ss._values = {"type": ts.selections}
ss

> TODO `ts.selections` still being rewritten to a list of strings. 

In [ ]:
ge_selector = GraphExploreNodeSelection()
ge_selector.graph = lw.graph
ge_selector

In [ ]:
ge_selector.subject_filter.selections

For now, select `A New Hope`, `C-3PO`, and `R2-D2`.

### Pass to RDF2NX

In [ ]:
from ipyradiant.rdf2nx import RDF2NX

nx_graph = RDF2NX.convert_nodes(node_uris=ge_selector.subject_filter.selections, rdf_graph=lw.graph)

In [ ]:
import json

import ipycytoscape
import ipywidgets as W

In [ ]:
directed = ipycytoscape.CytoscapeWidget()
directed.graph.add_graph_from_networkx(nx_graph, multiple_edges=True, directed=True)

In [ ]:
for node in directed.graph.nodes:
    # deal with inability to handle `:` e.g. thing:data
    node.data["_label"] = node.data.get("rdfs:label", None)
    node.data["_attrs"] = json.dumps(node.data, indent=2)

In [ ]:
directed.set_layout(
    name="dagre", animate=False, randomize=False, maxSimulationTime=2000
)
# Workaround for style overwriting
directed.set_style(
    [
        {
            "selector": "node",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "blue",
            },
        },
        {
            "selector": "edge",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "green",
            },
        },
        {
            "selector": "edge.directed",
            "style": {
                "curve-style": "bezier",
                "target-arrow-shape": "triangle",
            },
        },
        {"selector": "edge.multiple_edges", "style": {"curve-style": "bezier"}},
        {
            "selector": ":active ",
            "css": {
                "label": "data(_attrs)",
                "text-wrap": "wrap",
                "text-max-width": "500px",
                "text-valign": "bottom",
                "text-halign": "right",
                "text-background-opacity": 0.9,
                "text-background-color": "white",
                "text-background-shape": "roundrectangle",
                "color": "black",
            },
        },
    ]
)

In [ ]:
directed